# IEEE Card Fraud Detection

## Contents
1. Load Data
2. Exploratory Data Analysis
3. Building the base model
4. Building the tuned model

## 1. Load Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_id = pd.read_csv("data/train_identity.csv")
train_txn = pd.read_csv("data/train_transaction.csv")

print(f"Shape of train identity: {train_id.shape}")
print(f"Shape of train transaction: {train_txn.shape}")

Shape of train identity: (144233, 41)
Shape of train transaction: (590540, 394)


In [3]:
test_id = pd.read_csv("data/test_identity.csv")
test_txn = pd.read_csv("data/test_transaction.csv")

In [4]:
# primary key is TransactionID
train_df = pd.merge(train_txn,train_id, how = 'left', on = 'TransactionID',validate = "many_to_one")
test_df = pd.merge(test_txn,test_id, how = 'left', on = 'TransactionID',validate = "many_to_one")

In [5]:
train_df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


## Tune Model

In [6]:
# Importing the Keras libraries and packages
from sklearn.metrics import auc
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

#### Take a small subset for gridsearch

In [7]:
train_df_clean = train_df.sample(20000)

#### Split data

In [8]:
# train_txn = pd.read_csv("data/train_transaction.csv", nrows=10000)
X_train = train_df_clean.drop("isFraud", axis=1)
y_train = train_df_clean["isFraud"]

In [9]:
X_train.shape

(20000, 433)

In [10]:
y_train.shape

(20000,)

#### Remove irrelevant columns

In [11]:
col_to_drop = ['TransactionID']
for col in col_to_drop:
    X_train.drop(col, axis=1, inplace=True)

#### Handle categorical variables
We will onehotencode all categorical data that are strings (e.g. product type), and label encode the rest. `cat_cols` comes from the data description: https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203#latest-614363

In [12]:
cat_cols = ['ProductCD',
            'card1', 'card2','card3','card4','card5','card6', 
            'addr1', 'addr2', 
            'Pemaildomain', 'Remaildomain', 
            'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']

cat_cols_to_encode = [col for col in cat_cols if col in list(X_train)]

In [13]:
onehot_cols = [col for col in cat_cols_to_encode if X_train[col].dtype==object]
# Check that the cols for one hot encoding dont have too many unique values (to avoid sparse matrix)
for col in onehot_cols:
    num_unique = len(X_train[col].unique())
    print(f"{col} has {num_unique} unique values")
    
label_cols = [col for col in cat_cols_to_encode if col not in onehot_cols]

ProductCD has 5 unique values
card4 has 5 unique values
card6 has 5 unique values
M1 has 2 unique values
M2 has 3 unique values
M3 has 3 unique values
M4 has 4 unique values
M5 has 3 unique values
M6 has 3 unique values
M7 has 3 unique values
M8 has 3 unique values
M9 has 3 unique values


#### Label encode

In [14]:
def encode_features(data):
    for f in data.columns:
        if data[f].dtype=='object': 
            lbl = LabelEncoder()
            lbl.fit(list(data[f].values))
            data[f] = lbl.transform(list(data[f].values))
    
    print(f"Shape after encoding categorical variables: {data.shape}]")
    
    return data

In [15]:
X_train = encode_features(X_train)

Shape after encoding categorical variables: (20000, 432)]


### Train

In [16]:
import xgboost as xgb

In [17]:
# Check lenggth of x and y
if X_train.shape[0] == y_train.shape[0]:
    print("Number of rows MATCH between x and y")
else:
    print("Number of rows DO NOT MATCH between x and y")

Number of rows MATCH between x and y


In [18]:
xgb_model = xgb.XGBClassifier()
optimization_dict = {'max_depth': [5, 10, 15, 20],
                     'min_child_weight': [1, 3, 5],
                     'max_delta_step': [0, 1], # for imbalanced class
                     'learning_rate': [0.1, 0.],
                     'n_estimators': [200, 500, 1000]}


In [ ]:
tuned_model = GridSearchCV(xgb_model, optimization_dict, 
                           scoring='accuracy', verbose=1)

tuned_model.fit(X_train, y_train)
print(tuned_model.best_score_)
print(tuned_model.best_params_)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


C:\Users\himi6\AppData\Local\Continuum\anaconda3\envs\deep_learning\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [39]:
# Save model
import pickle
pickle.dump(base_model, open("ieee_fraud_no_drop_nans.pickle", "wb"))

### Test

In [40]:
X_test = encode_features(test_df)

Shape after encoding categorical variables: (506691, 433)]


In [41]:
X_test

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3663549,18403224,31.950,4,10409,111.0,150.0,4,226.0,2,...,107,NaN,390,2,2,2,2,2,2,2184
1,3663550,18403263,49.000,4,4272,111.0,150.0,4,226.0,2,...,107,NaN,390,2,2,2,2,2,2,2184
2,3663551,18403310,171.000,4,4476,574.0,150.0,4,226.0,2,...,107,NaN,390,2,2,2,2,2,2,2184
3,3663552,18403310,284.950,4,10989,360.0,150.0,4,166.0,2,...,107,NaN,390,2,2,2,2,2,2,2184
4,3663553,18403317,67.950,4,18018,452.0,150.0,2,117.0,2,...,107,NaN,390,2,2,2,2,2,2,2184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,4170235,34214279,94.679,0,13832,375.0,185.0,2,224.0,2,...,107,NaN,390,2,2,2,2,2,2,2184
506687,4170236,34214287,12.173,0,3154,408.0,185.0,2,224.0,2,...,5,NaN,390,2,0,0,1,0,1,107
506688,4170237,34214326,49.000,4,16661,490.0,150.0,4,226.0,2,...,107,NaN,390,2,2,2,2,2,2,2184
506689,4170238,34214337,202.000,4,16621,516.0,150.0,2,224.0,2,...,107,NaN,390,2,2,2,2,2,2,2184


In [42]:
txn_id_list = X_test['TransactionID']
col_to_drop = ['TransactionID']
for col in col_to_drop:
    X_test.drop(col, axis=1, inplace=True)
    
X_test = X_test[list(X_train)]

In [43]:
y_pred = base_model.predict_proba(X_test)[:,1] 

In [44]:
my_submission = pd.DataFrame({"TransactionID": txn_id_list,
                              "isFraud": y_pred})

In [45]:
my_submission

,TransactionID,isFraud
0,3663549,0.003448
1,3663550,0.007339
2,3663551,0.021096
3,3663552,0.002765
4,3663553,0.007624
...,...,...
506686,4170235,0.023764
506687,4170236,0.060578
506688,4170237,0.010602
506689,4170238,0.018641


In [46]:
my_submission.to_csv("submission_didnt_drop_nans.csv", index=False)

## Feature Engineering
Potential Features
* Time of day compared to normal
* Time between transactions
* If the same transaction was just made/attempted

## Model Training